In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/skku/2021-1 Machine Learning/ml project/train.csv")

In [ ]:
df.head()

,emotion_angry_mean,emotion_disgust_mean,emotion_fear_mean,emotion_happy_mean,emotion_sad_mean,emotion_surprise_mean,emotion_neutral_mean,F0semitoneFrom27.5Hz_sma3nz_amean_mean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm_mean,F0semitoneFrom27.5Hz_sma3nz_percentile20.0_mean,F0semitoneFrom27.5Hz_sma3nz_percentile50.0_mean,F0semitoneFrom27.5Hz_sma3nz_percentile80.0_mean,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2_mean,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope_mean,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope_mean,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope_mean,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope_mean,loudness_sma3_amean_mean,loudness_sma3_stddevNorm_mean,loudness_sma3_percentile20.0_mean,loudness_sma3_percentile50.0_mean,loudness_sma3_percentile80.0_mean,loudness_sma3_pctlrange0-2_mean,loudness_sma3_meanRisingSlope_mean,loudness_sma3_stddevRisingSlope_mean,loudness_sma3_meanFallingSlope_mean,loudness_sma3_stddevFallingSlope_mean,spectralFlux_sma3_amean_mean,spectralFlux_sma3_stddevNorm_mean,mfcc1_sma3_amean_mean,mfcc1_sma3_stddevNorm_mean,mfcc2_sma3_amean_mean,mfcc2_sma3_stddevNorm_mean,mfcc3_sma3_amean_mean,mfcc3_sma3_stddevNorm_mean,mfcc4_sma3_amean_mean,mfcc4_sma3_stddevNorm_mean,jitterLocal_sma3nz_amean_mean,jitterLocal_sma3nz_stddevNorm_mean,shimmerLocaldB_sma3nz_amean_mean,...,F2bandwidth_sma3nz_stddevNorm_max,F2amplitudeLogRelF0_sma3nz_amean_max,F2amplitudeLogRelF0_sma3nz_stddevNorm_max,F3frequency_sma3nz_amean_max,F3frequency_sma3nz_stddevNorm_max,F3bandwidth_sma3nz_amean_max,F3bandwidth_sma3nz_stddevNorm_max,F3amplitudeLogRelF0_sma3nz_amean_max,F3amplitudeLogRelF0_sma3nz_stddevNorm_max,alphaRatioV_sma3nz_amean_max,alphaRatioV_sma3nz_stddevNorm_max,hammarbergIndexV_sma3nz_amean_max,hammarbergIndexV_sma3nz_stddevNorm_max,slopeV0-500_sma3nz_amean_max,slopeV0-500_sma3nz_stddevNorm_max,slopeV500-1500_sma3nz_amean_max,slopeV500-1500_sma3nz_stddevNorm_max,spectralFluxV_sma3nz_amean_max,spectralFluxV_sma3nz_stddevNorm_max,mfcc1V_sma3nz_amean_max,mfcc1V_sma3nz_stddevNorm_max,mfcc2V_sma3nz_amean_max,mfcc2V_sma3nz_stddevNorm_max,mfcc3V_sma3nz_amean_max,mfcc3V_sma3nz_stddevNorm_max,mfcc4V_sma3nz_amean_max,mfcc4V_sma3nz_stddevNorm_max,alphaRatioUV_sma3nz_amean_max,hammarbergIndexUV_sma3nz_amean_max,slopeUV0-500_sma3nz_amean_max,slopeUV500-1500_sma3nz_amean_max,spectralFluxUV_sma3nz_amean_max,loudnessPeaksPerSec_max,VoicedSegmentsPerSec_max,MeanVoicedSegmentLengthSec_max,StddevVoicedSegmentLengthSec_max,MeanUnvoicedSegmentLength_max,StddevUnvoicedSegmentLength_max,equivalentSoundLevel_dBp_max,label
0,0.159302,0.001206,0.095556,0.103968,0.241841,0.015937,0.380635,24.365636,0.175620,21.451660,23.824941,26.860989,5.409329,178.302275,118.805772,71.780979,60.595704,1.120236,0.456544,0.679286,1.030189,1.554274,0.874988,11.056342,5.862386,7.338043,4.313646,0.626464,0.695773,21.477432,0.693588,-0.327011,2.948707,2.301753,0.621157,-0.636670,1.357991,0.041140,1.294767,1.411157,...,0.013287,1553.790611,13.057997,17276.461014,0.000760,15273.899997,0.010718,1454.616107,0.333077,11.872866,0.153309,17.167801,0.121265,0.000749,2.103438e+01,0.000062,2.359656,0.082971,0.056775,38.786267,0.067508,39.441458,166082.362816,76.087625,93.992936,77.820486,36532.756730,19.835802,40.867942,0.001236,0.000041,0.073747,3.247842,1.524696,0.038395,0.007741,0.009176,0.002994,17.493596,0
1,0.078909,0.000098,0.097341,0.111471,0.152525,0.038076,0.521042,20.450413,0.180559,17.321590,20.874958,23.396257,6.074668,137.679613,87.802155,67.882583,52.059239,0.622723,0.416345,0.396978,0.559901,0.836382,0.439405,4.619568,2.419374,2.571888,1.489682,0.307432,0.502883,25.386675,0.305130,2.699263,1.668620,4.992532,1.532773,3.519948,2.028082,0.065575,1.384110,1.419527,...,0.005533,491.589328,0.276725,20300.495507,0.000696,10802.614674,0.008120,445.160892,0.125321,4.298156,0.011688,8.891616,0.006371,0.000527,1.106866e+07,0.000032,212.740090,0.012751,0.033280,24.012308,0.017846,21.320284,928.943139,41.933796,520.846650,41.720618,388.881196,15.238695,47.110117,0.000494,0.000053,

In [ ]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1:]

In [ ]:
print(X.shape)

(405, 383)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape) # 데이터 개수 확인

(324, 383) (81, 383) (324, 1) (81, 1)


In [ ]:
X_train.to_numpy().shape

(324, 383)

In [ ]:
import random
import time
import progressbar

In [ ]:
def gini_impurity(labels):
    # When the set is empty, it is also pure
    if labels.size==0:
        return 0
    
    counts = np.unique(labels, return_counts=True)[1]

    fractions = counts / float(len(labels))
    
    return 1 - np.sum(fractions ** 2)

In [ ]:
def weighted_impurity(groups):
    """ Calculate weighted impurity of children after a split
    Args:
        groups (list of children, and a child consists a list of class labels)
    Returns:
        float, weighted impurity
    """
    total = sum(len(group) for group in groups)
    weighted_sum = 0.0
    for group in groups:
        weighted_sum += len(group) / float(total) * gini_impurity(group)
    return weighted_sum

In [ ]:
def split_node(X, y, index, value):
    """ Split data set X, y based on a feature and a value
    Args:
        X, y (numpy.ndarray, data set)
        index (int, index of the feature used for splitting)
        value (value of the feature used for splitting)
    Returns:
        list, list: left and right child, a child is in the format of [X, y]
    """
    
    left = [np.array([x for x in X if x[index] < value]), np.array([y[i] for i, x in enumerate(X) if x[index] < value])]
    right = [np.array([x for x in X if x[index] >= value]), np.array([y[i] for i, x in enumerate(X) if x[index] >= value])]

    return left, right

In [ ]:
def get_best_split(X, y, features):
    """ Obtain the best splitting point and resulting children for the data set X, y
    Args:
        X, y (numpy.ndarray, data set)
    Returns:
        list [index, value, children]
    """
    best_index, best_value, best_score, children = None, None, 1, None
    for index in features:
        for value in np.sort(np.unique(X[:, index])):
            children = split_node(X, y, index, value)
            score = weighted_impurity([children[0][1], children[1][1]])
            if score < best_score:
                best_index = index
                best_value = value
                best_score = score
    children = split_node(X, y, best_index, best_value)

    return [best_index, best_value, children]

In [ ]:
def get_leaf(labels):
    # Obtain the leaf as the majority of the labels
    return np.bincount(labels).argmax()

In [ ]:
def split(node, max_depth, min_size, depth, i):
    """ Split children of a node to construct new nodes or assign them terminals
    Args:
        node (list, with children info)
        max_depth (int, maximal depth of the tree)
        min_size (int, minimal samples required to further split a child)
        depth (int, current depth of the node)
        i (int, location in node)
    """
    left, right = node[i][2]
    del (node[i][2])
    if left[1].size == 0:
        node[2*i+2] = [get_leaf(right[1])]
        return
    if right[1].size == 0:
        node[2*i+1] = [get_leaf(left[1])]
        return
    # Check if the current depth exceeds the maximal depth
    if depth >= max_depth:
        node[2*i+1], node[2*i+2] = [get_leaf(left[1])], [get_leaf(right[1])]
        return
    # Check if the left child has enough samples
    if left[1].size <= min_size:
        node[2*i+1] = [get_leaf(left[1])]
    else:
        # It has enough samples, we further split it
        random_features = random.sample(range(0,383),20)
        node[2*i+1] = get_best_split(left[0], left[1], random_features)
        split(node, max_depth, min_size, depth + 1, 2*i+1)

    # Check if the right child has enough samples
    if right[1].size <= min_size:
        node[2*i+2] = [get_leaf(right[1])]
    else:
        # It has enough samples, we further split it
        random_features = random.sample(range(0,383),20)
        node[2*i+2] = get_best_split(right[0], right[1], random_features)
        split(node, max_depth, min_size, depth + 1, 2*i+2)

In [ ]:
class RandomForest():
    def __init__(self, n_estimators=100, max_features=383, max_depth=4, min_size=18):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_size = min_size
        self.progressbar = progressbar.ProgressBar()

        self.trees = []

    def fit(self, X, y):
        for _ in self.progressbar(range(self.n_estimators)):
            data_index = [data_index for data_index in range(X.shape[0])] #324
            random_data_index = np.random.choice(data_index, X.shape[0]) # 복원추출
            
            X_ = np.array([X[data] for data in random_data_index])
            y_ = np.array([y[data] for data in random_data_index])
        
            node = [0 for _ in range(2**(self.max_depth+1)-1)]
            random_features = random.sample(range(0,self.max_features),20)
            node[0] = get_best_split(X_, y_, random_features)
            split(node, self.max_depth, self.min_size, 1, 0)
            self.trees.append(node)

    def predict(self, X):
        y_preds = np.empty((X.shape[0], len(self.trees))) #81, n_estimators
        for i, tree in enumerate(self.trees): #n_estimators
            check = [] # 0 or 1
            
            for x in X: #81test
                j=0
                while (tree[j]!=[0] and tree[j]!=[1]):
                    if x[tree[j][0]] < tree[j][1]: #left
                        j = 2*j+1
                        if tree[j]==0:
                            j = j+1
                    else: #right
                        j = 2*j+2
                        if tree[j]==0:
                            j = j-1
                check.append(tree[j][0]) # 0 or 1
                
            y_preds[:, i] = np.array(check)
        
        y_pred = []
        for prediction in y_preds:
            y_pred.append(np.bincount(prediction.astype('int')).argmax())

        return y_pred

In [ ]:
baseline = RandomForest()
baseline.fit(X_train.to_numpy(), y_train.to_numpy().ravel())

100% (100 of 100) |######################| Elapsed Time: 0:15:49 Time:  0:15:49


In [ ]:
y_pred = baseline.predict(X_test.to_numpy())

In [ ]:
for y in y_pred:
    print(y, end=' ')
print("\n result // 0 : ", y_pred.count(0), ", 1 : ", y_pred.count(1))

0 0 0 0 0 0 1 0 0 0 1 1 0 1 0 1 1 0 1 1 0 0 0 0 1 0 1 1 0 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 0 0 0 0 0 1 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 1 0 
 result // 0 :  47 , 1 :  34


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.76      0.78        50
           1       0.65      0.71      0.68        31

    accuracy                           0.74        81
   macro avg       0.73      0.73      0.73        81
weighted avg       0.75      0.74      0.74        81



In [ ]:
final = pd.read_csv('/content/drive/MyDrive/skku/2021-1 Machine Learning/ml project 1/X_test.csv')

In [ ]:
final.head()

,emotion_angry_mean,emotion_disgust_mean,emotion_fear_mean,emotion_happy_mean,emotion_sad_mean,emotion_surprise_mean,emotion_neutral_mean,F0semitoneFrom27.5Hz_sma3nz_amean_mean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm_mean,F0semitoneFrom27.5Hz_sma3nz_percentile20.0_mean,F0semitoneFrom27.5Hz_sma3nz_percentile50.0_mean,F0semitoneFrom27.5Hz_sma3nz_percentile80.0_mean,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2_mean,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope_mean,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope_mean,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope_mean,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope_mean,loudness_sma3_amean_mean,loudness_sma3_stddevNorm_mean,loudness_sma3_percentile20.0_mean,loudness_sma3_percentile50.0_mean,loudness_sma3_percentile80.0_mean,loudness_sma3_pctlrange0-2_mean,loudness_sma3_meanRisingSlope_mean,loudness_sma3_stddevRisingSlope_mean,loudness_sma3_meanFallingSlope_mean,loudness_sma3_stddevFallingSlope_mean,spectralFlux_sma3_amean_mean,spectralFlux_sma3_stddevNorm_mean,mfcc1_sma3_amean_mean,mfcc1_sma3_stddevNorm_mean,mfcc2_sma3_amean_mean,mfcc2_sma3_stddevNorm_mean,mfcc3_sma3_amean_mean,mfcc3_sma3_stddevNorm_mean,mfcc4_sma3_amean_mean,mfcc4_sma3_stddevNorm_mean,jitterLocal_sma3nz_amean_mean,jitterLocal_sma3nz_stddevNorm_mean,shimmerLocaldB_sma3nz_amean_mean,...,F2bandwidth_sma3nz_amean_max,F2bandwidth_sma3nz_stddevNorm_max,F2amplitudeLogRelF0_sma3nz_amean_max,F2amplitudeLogRelF0_sma3nz_stddevNorm_max,F3frequency_sma3nz_amean_max,F3frequency_sma3nz_stddevNorm_max,F3bandwidth_sma3nz_amean_max,F3bandwidth_sma3nz_stddevNorm_max,F3amplitudeLogRelF0_sma3nz_amean_max,F3amplitudeLogRelF0_sma3nz_stddevNorm_max,alphaRatioV_sma3nz_amean_max,alphaRatioV_sma3nz_stddevNorm_max,hammarbergIndexV_sma3nz_amean_max,hammarbergIndexV_sma3nz_stddevNorm_max,slopeV0-500_sma3nz_amean_max,slopeV0-500_sma3nz_stddevNorm_max,slopeV500-1500_sma3nz_amean_max,slopeV500-1500_sma3nz_stddevNorm_max,spectralFluxV_sma3nz_amean_max,spectralFluxV_sma3nz_stddevNorm_max,mfcc1V_sma3nz_amean_max,mfcc1V_sma3nz_stddevNorm_max,mfcc2V_sma3nz_amean_max,mfcc2V_sma3nz_stddevNorm_max,mfcc3V_sma3nz_amean_max,mfcc3V_sma3nz_stddevNorm_max,mfcc4V_sma3nz_amean_max,mfcc4V_sma3nz_stddevNorm_max,alphaRatioUV_sma3nz_amean_max,hammarbergIndexUV_sma3nz_amean_max,slopeUV0-500_sma3nz_amean_max,slopeUV500-1500_sma3nz_amean_max,spectralFluxUV_sma3nz_amean_max,loudnessPeaksPerSec_max,VoicedSegmentsPerSec_max,MeanVoicedSegmentLengthSec_max,StddevVoicedSegmentLengthSec_max,MeanUnvoicedSegmentLength_max,StddevUnvoicedSegmentLength_max,equivalentSoundLevel_dBp_max
0,0.155693,0.000225,0.077640,0.226517,0.208652,0.044869,0.285805,22.734337,0.204727,18.768689,22.825144,26.437933,7.669243,154.216004,120.644116,98.211368,75.855315,1.015353,0.517132,0.572970,0.892117,1.437425,0.864454,11.547967,6.075034,8.175721,4.964917,0.838042,0.816384,24.897853,0.527930,1.689141,-3.422121,14.106929,0.737629,3.275405,3.215152,0.076818,1.205838,1.659669,...,17982.470382,0.012680,616.083640,0.061312,6200.688869,0.000911,15808.225844,0.012500,591.961803,0.049908,7.843710,0.022597,14.797768,0.013479,0.000349,4330.337793,0.000049,0.019668,0.138786,0.069729,21.824390,0.018730,30.531802,307.571620,66.268935,268.242701,50.879484,667.047326,22.063426,34.789545,0.000287,0.000038,0.051829,2.445959,2.175879,0.027020,0.005952,0.003459,0.001186,15.198562
1,0.177189,0.000472,0.095150,0.250708,0.331609,0.024785,0.118970,32.708595,0.124259,29.709912,32.511974,35.718257,6.008345,136.941469,83.734519,74.981479,57.387060,1.922800,0.456558,1.177125,1.734119,2.672978,1.495853,18.064664,9.747658,13.769853,8.826276,1.329625,0.712600,21.362207,0.830010,-17.246966,-0.134327,15.995365,1.108757,-15.705268,-13.643544,0.036286,1.259429,1.129771,...,50304.793142,0.019419,2070.233673,34.576160,143758.858265,0.000785,42272.420261,0.015091,1995.585941,1.786308,19.838539,7821.411711,27.573449,4.331645,0.001032,4.839315,0.000077,7.919872,0.517096,0.040509,71.631249,0.293790,94.483504,52.795820,88.068110,106.725353,114.730813

In [ ]:
y_pred = baseline.predict(final.to_numpy())

In [ ]:
for y in y_pred:
    print(y, end=' ')
print("\n0 : ", y_pred.count(0), ", 1 : ", y_pred.count(1))

1 0 1 1 1 1 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 0 1 0 0 1 1 1 1 1 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 0 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 0 0 1 0 1 1 1 
0 :  49 , 1 :  51


In [ ]:
pd.DataFrame(y_pred).to_csv("2019313464.csv", header=None) #2019313464